<a href="https://colab.research.google.com/github/UBAILAB/Study/blob/main/Chap01_07_Regularized_Model_Ridge_Code_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color=white> 07.Regularized Model-Ridge Code 실습**

[목적]
  1. Linear Regression
    - 단순 Linear Regression을 활용하여 변수의 중요도 및 방향성을 알아봄
    - 매우 심플한 모델이기 때문에 사이즈가 큰 데이터에 적합하지 않음
    - 하지만 설명력에서는 큰 장점이 있음
  2. Ridge Regression
    - Regularized Linear Model을 활용하여 Overfitting을 방지함
    - Hyperparameter lamba를 튜닝할 때 for loop 뿐만 아니라 GridsearchCV를 통해 돌출해봄
  3. Regularized Linear Models의 경우 X's Scaling을 필수적으로 진행해야함

[Process]
  1. Define X's & Y
  2. Split Train & Valid dataset
  3. Modeling
  4. Model 해석

In [1]:
# Package
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Sklearn Toy Data
from sklearn.datasets import load_diabetes

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes

In [2]:
# Data Loading (당뇨병)
data = load_diabetes()

In [6]:
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [3]:
print('>>>>X Shape : {}'.format(data['data'].shape))
print('>>>>Y Shape : {}'.format(data['target'].shape))

>>>>X Shape : (442, 10)
>>>>Y Shape : (442,)


In [8]:
print('X Shape : {}'.format(data['data'].shape))
print('Y Shape : {}'.format(data['target'].shape))

X Shape : (442, 10)
Y Shape : (442,)


In [9]:
# 데이터 설명
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [10]:
pd.DataFrame(data['data'], columns=data['feature_names']).describe().applymap(lambda x:f"{x:0.2f}")

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00
mean,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00
std,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
min,-0.11,-0.04,-0.09,-0.11,-0.13,-0.12,-0.10,-0.08,-0.13,-0.14
25%,-0.04,-0.04,-0.03,-0.04,-0.03,-0.03,-0.04,-0.04,-0.03,-0.03
50%,0.01,-0.04,-0.01,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00
75%,0.04,0.05,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03
max,0.11,0.05,0.17,0.13,0.15,0.20,0.18,0.19,0.13,0.14


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('age').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('sex').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('bmi').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('bp').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['sex'].value_counts()
    for x_label, grp in _df_4.groupby('age')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('age')
_ = plt.ylabel('sex')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['bmi'].value_counts()
    for x_label, grp in _df_5.groupby('sex')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('sex')
_ = plt.ylabel('bmi')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['bp'].value_counts()
    for x_label, grp in _df_6.groupby('bmi')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('bmi')
_ = plt.ylabel('bp')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['s1'].value_counts()
    for x_label, grp in _df_7.groupby('bp')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('bp')
_ = plt.ylabel('s1')

In [ ]:
pd.DataFrame(data['data'], columns=data['feature_names']).describe().applymap(lambda x: f"{x:0.2f}")

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00
mean,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00
std,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
min,-0.11,-0.04,-0.09,-0.11,-0.13,-0.12,-0.10,-0.08,-0.13,-0.14
25%,-0.04,-0.04,-0.03,-0.04,-0.03,-0.03,-0.04,-0.04,-0.03,-0.03
50%,0.01,-0.04,-0.01,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00
75%,0.04,0.05,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03
max,0.11,0.05,0.17,0.13,0.15,0.20,0.18,0.19,0.13,0.14


In [12]:
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

In [11]:
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

In [13]:
data.describe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [14]:
# X's & Y Split
Y = data['Y']
X = data.drop(columns=['Y'])
X = pd.get_dummies(X, columns=['SEX'])

In [15]:
Y = data['Y']
X = data.drop(columns = ['Y'])
X = pd.get_dummies(X, columns=['SEX'])

In [16]:
X

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,59,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,0,1
1,48,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,1,0
2,72,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,0,1
3,24,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,1,0
4,50,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,1,0
...,...,...,...,...,...,...,...,...,...,...,...
437,60,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,0,1
438,47,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,0,1
439,60,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,0,1
440,36,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,1,0


[Data Split]

- Data Split을 진행할 때 BigData의 경우 꼭 indexing을 추출하여 모델에 적용시켜야 함
- 이유는 Data Split하여 새로운 Data set을 만들 경우 메모리에 부담을 주기 때문

In [17]:
X.shape[0]

442

In [19]:
idx = list(range(X.shape[0]))
idx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [21]:
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(f"Train data : {len(train_idx)}")
print(f"Valid data : {len(valid_idx)}")

Train data : 309
Valid data : 133


In [20]:
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

>>>> # of Train data : 309
>>>> # of valid data : 133


In [22]:
train_idx

[233,
 373,
 341,
 92,
 203,
 100,
 423,
 320,
 175,
 252,
 245,
 385,
 408,
 426,
 40,
 337,
 27,
 188,
 190,
 235,
 102,
 109,
 167,
 405,
 272,
 11,
 286,
 17,
 410,
 118,
 130,
 150,
 217,
 161,
 411,
 262,
 149,
 330,
 314,
 398,
 116,
 256,
 397,
 338,
 282,
 126,
 377,
 236,
 438,
 57,
 309,
 248,
 260,
 44,
 264,
 43,
 208,
 184,
 147,
 277,
 231,
 386,
 323,
 151,
 83,
 274,
 316,
 46,
 159,
 415,
 112,
 187,
 2,
 81,
 267,
 266,
 8,
 35,
 79,
 393,
 140,
 88,
 12,
 382,
 180,
 223,
 399,
 347,
 326,
 226,
 120,
 271,
 91,
 93,
 74,
 402,
 345,
 185,
 221,
 224,
 437,
 31,
 21,
 360,
 144,
 335,
 96,
 406,
 261,
 197,
 243,
 168,
 334,
 95,
 358,
 346,
 76,
 107,
 336,
 214,
 306,
 276,
 7,
 38,
 353,
 50,
 138,
 247,
 23,
 142,
 99,
 370,
 391,
 212,
 73,
 82,
 228,
 145,
 237,
 342,
 270,
 268,
 59,
 108,
 428,
 113,
 179,
 367,
 204,
 308,
 218,
 211,
 374,
 5,
 164,
 222,
 283,
 192,
 60,
 45,
 163,
 173,
 117,
 215,
 425,
 48,
 98,
 29,
 186,
 14,
 85,
 10,
 106,
 105,
 1

In [ ]:
# Linear Regression
results = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])

In [23]:
results = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])

In [24]:
import scipy
from sklearn import metrics

def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

In [25]:
summary(results, X.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept -353.422717  3.804036e+08+7.000000e-06j -0.0000+0.0000j  0.999999
AGE          -0.241046  3.422080e-01+4.798600e-02j -0.6908+0.0969j  0.486679
BMI           5.364734  1.263233e+00+3.469000e-03j  4.2468-0.0117j  0.000041
BP            0.973515  3.572040e-01+5.036500e-02j  2.6723-0.3768j  0.007872
S1           -1.128987  3.352920e-01+3.755700e-02j -3.3255+0.3725j  0.001067
S2            0.935342  3.647620e-01+8.590000e-04j  2.5642-0.0060j  0.011459
S3            0.295834  4.142180e-01+1.898200e-02j  0.7127-0.0327j  0.476827
S4            2.577375  1.011969e+01+1.900000e-05j  0.2547-0.0000j  0.799360
S5           72.840272  2.200654e+01+1.290000e-04j  3.3099-0.0000j  0.001203
S6            0.292290  3.809690e-01+4.572100e-02j  0.7563-0.0908j  0.447561
SEX_1        10.444984  3.804036e+08+3.000000e-06j  0.0000-0.0000j  1.000000
SEX_2       -10.444984  3.804036e+08-2.000000e-06j -0.0000-0.0

In [28]:
summary(results, X.iloc[train_idx], Y.iloc[train_idx], xlabels=X.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept -353.422717  2.090546e+08-0.000000e+00j -0.0000-0.0000j  0.999999
AGE          -0.241046  2.810350e-01+0.000000e+00j -0.8577+0.0000j  0.391721
BMI           5.364734  7.773050e-01-0.000000e+00j  6.9017+0.0000j  0.000000
BP            0.973515  2.299340e-01-0.000000e+00j  4.2339+0.0000j  0.000030
S1           -1.128987  2.052450e-01+0.000000e+00j -5.5007+0.0000j  0.000000
S2            0.935342  2.423150e-01+0.000000e+00j  3.8600-0.0000j  0.000138
S3            0.295834  2.778100e-01+0.000000e+00j  1.0649-0.0000j  0.287765
S4            2.577375  6.167304e+00-0.000000e+00j  0.4179+0.0000j  0.676304
S5           72.840272  1.338508e+01-0.000000e+00j  5.4419+0.0000j  0.000000
S6            0.292290  2.721370e-01-0.000000e+00j  1.0741+0.0000j  0.283638
SEX_1        10.444984  2.090546e+08+0.000000e+00j  0.0000-0.0000j  1.000000
SEX_2       -10.444984  2.090546e+08-0.000000e+00j -0.0000-0.0

In [31]:
# Scaling
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)

In [30]:
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns = X.columns)

In [32]:
X_scal

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,0.666667,0.573840,0.565217,0.294118,0.297578,0.197368,0.318471,0.562217,0.439394,0.0,1.0
1,0.483333,0.130802,0.362319,0.421569,0.355248,0.618421,0.159236,0.222437,0.166667,1.0,0.0
2,0.883333,0.506329,0.449275,0.289216,0.299885,0.236842,0.318471,0.496578,0.409091,0.0,1.0
3,0.083333,0.286920,0.318841,0.495098,0.517878,0.223684,0.477707,0.572923,0.469697,1.0,0.0
4,0.516667,0.189873,0.565217,0.465686,0.483276,0.381579,0.318471,0.362385,0.333333,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.683333,0.409283,0.724638,0.431373,0.416378,0.250000,0.318471,0.605672,0.530303,0.0,1.0
438,0.466667,0.270042,0.188406,0.627451,0.717416,0.250000,0.477707,0.415810,0.666667,0.0,1.0
439,0.683333,0.270042,0.545942,0.318627,0.374856,0.263158,0.281847,0.305030,0.560606,0.0,1.0
440,0.283333,0.485232,0.478261,0.509804,0.482122,0.250000,0.444268,0.657026,0.409091,1.0,0.0


In [33]:
# Linear Regression
results = LinearRegression().fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
summary(results, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scal.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   -2.765884  3.804036e+08+2.690939e+00j -0.0000+0.0000j  1.000000
AGE         -14.462769  2.374695e+01+2.592370e-01j -0.6090+0.0066j  0.543571
BMI         127.144195  3.179312e+01+1.736110e-01j  3.9990-0.0218j  0.000105
BP           67.172560  2.807443e+01+3.545510e-01j  2.3923-0.0302j  0.018144
S1         -230.313267  1.636917e+02+3.033620e-01j -1.4070+0.0026j  0.161780
S2          162.188278  1.149954e+02+9.065050e-01j  1.4103-0.0111j  0.160790
S3           22.483360  7.291746e+01-2.444690e-01j  0.3083+0.0010j  0.758311
S4           16.185916  5.719609e+01+2.402260e-01j  0.2830-0.0012j  0.777630
S5          207.514650  5.030871e+01+6.675700e-02j  4.1248-0.0055j  0.000065
S6           19.291168  3.403224e+01+4.295500e-01j  0.5668-0.0072j  0.571809
SEX_1        10.444984  3.804036e+08+1.864951e+00j  0.0000-0.0000j  1.000000
SEX_2       -10.444984  3.804036e+08+6.892400e-02j -0.0000+0.0

[Ridge Regression]
 - Hyperparameter Tuning using for Loop
 - Hyperparameter Tuning using GridSearchCV

[Ridge Regression Parameters]
   - Package : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
   - alpha : L2-norm Penalty Term
     - alpha : 0 일 때, Just Linear Regression
   - fit_intercept : Centering to zero
     - 베타0를 0로 보내는 것 (베타0는 상수이기 때문에)
   - max_iter : Maximum number of interation
     - Loss Function의 Ridge Penalty Term은 Closed Form 값이기는 하지만 값을 찾아 나감
     - Penalty Term : (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_2

In [34]:
penelty = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9, 1, 10]

# Using For Loop !!
# Ridge Regression
# select alpha by checking R2, MSE, RMSE
for a in penelty:
    model = Ridge(alpha=a).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
    score = model.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
    pred_y = model.predict(X_scal.iloc[valid_idx])
    mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
    print("Alpha:{0:.5f}, R2:{1:.7f}, MSE:{2:.7f}, RMSE:{3:.7f}".format(a, score, mse, np.sqrt(mse)))

Alpha:0.00001, R2:0.5301655, MSE:3084.6095744, RMSE:55.5392616
Alpha:0.00005, R2:0.5301672, MSE:3084.5988322, RMSE:55.5391648
Alpha:0.00010, R2:0.5301692, MSE:3084.5854446, RMSE:55.5390443
Alpha:0.00100, R2:0.5302048, MSE:3084.3519225, RMSE:55.5369420
Alpha:0.01000, R2:0.5304637, MSE:3082.6522133, RMSE:55.5216373
Alpha:0.10000, R2:0.5304511, MSE:3082.7343410, RMSE:55.5223769
Alpha:0.30000, R2:0.5294946, MSE:3089.0142040, RMSE:55.5789007
Alpha:0.50000, R2:0.5285641, MSE:3095.1232917, RMSE:55.6338323
Alpha:0.60000, R2:0.5280578, MSE:3098.4477026, RMSE:55.6637018
Alpha:0.70000, R2:0.5275205, MSE:3101.9751863, RMSE:55.6953785
Alpha:0.90000, R2:0.5263592, MSE:3109.5989675, RMSE:55.7637783
Alpha:1.00000, R2:0.5257398, MSE:3113.6657269, RMSE:55.8002305
Alpha:10.00000, R2:0.4513724, MSE:3601.9109235, RMSE:60.0159222


In [47]:
penelty = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9,1,10]
for a in penelty:
  model = Ridge(alpha = a).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
  score = model.score(X_scal.iloc[valid_idx],Y.iloc[valid_idx])
  pred_y = model.predict(X_scal.iloc[valid_idx])
  mse=mean_squared_error(Y.iloc[valid_idx],pred_y)
  print(f"Alpha:{a:5f}, R2:{score:7f}, MSE:{mse:7f}, RMSE:{np.sqrt(mse):7f}")
  print(f"Alpha:{a:.5f}, R2:{score:.7f}, MSE:{mse:.7f}, RMSE:{np.sqrt(mse):.7f}")

Alpha:0.000010, R2:0.530166, MSE:3084.609574, RMSE:55.539262
Alpha:0.00001, R2:0.5301655, MSE:3084.6095744, RMSE:55.5392616
Alpha:0.000050, R2:0.530167, MSE:3084.598832, RMSE:55.539165
Alpha:0.00005, R2:0.5301672, MSE:3084.5988322, RMSE:55.5391648
Alpha:0.000100, R2:0.530169, MSE:3084.585445, RMSE:55.539044
Alpha:0.00010, R2:0.5301692, MSE:3084.5854446, RMSE:55.5390443
Alpha:0.001000, R2:0.530205, MSE:3084.351922, RMSE:55.536942
Alpha:0.00100, R2:0.5302048, MSE:3084.3519225, RMSE:55.5369420
Alpha:0.010000, R2:0.530464, MSE:3082.652213, RMSE:55.521637
Alpha:0.01000, R2:0.5304637, MSE:3082.6522133, RMSE:55.5216373
Alpha:0.100000, R2:0.530451, MSE:3082.734341, RMSE:55.522377
Alpha:0.10000, R2:0.5304511, MSE:3082.7343410, RMSE:55.5223769
Alpha:0.300000, R2:0.529495, MSE:3089.014204, RMSE:55.578901
Alpha:0.30000, R2:0.5294946, MSE:3089.0142040, RMSE:55.5789007
Alpha:0.500000, R2:0.528564, MSE:3095.123292, RMSE:55.633832
Alpha:0.50000, R2:0.5285641, MSE:3095.1232917, RMSE:55.6338323
Alpha:0.

In [61]:
model_best = Ridge(alpha=0.9).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels = X_scal.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   42.643650  3.819412e+08+1.308048e+00j  0.0000-0.0000j  1.000000
AGE         -10.492753  2.379306e+01+1.368090e-01j -0.4410+0.0025j  0.659939
BMI         121.677019  3.185560e+01+1.175510e-01j  3.8196-0.0141j  0.000205
BP           66.649695  2.809700e+01+2.136490e-01j  2.3720-0.0180j  0.019133
S1          -21.091765  1.642265e+02+2.031570e-01j -0.1284+0.0002j  0.898004
S2           -5.450923  1.152949e+02+5.286150e-01j -0.0473+0.0002j  0.962363
S3          -55.208206  7.328338e+01-1.523860e-01j -0.7533-0.0016j  0.452581
S4           17.858217  5.733621e+01+1.527930e-01j  0.3115-0.0008j  0.755939
S5          119.080753  5.044998e+01+5.800700e-02j  2.3604-0.0027j  0.019721
S6           22.603065  3.410336e+01+2.433040e-01j  0.6627-0.0047j  0.508637
SEX_1        10.489062  3.819412e+08+1.290408e+00j  0.0000-0.0000j  1.000000
SEX_2       -10.489062  3.819412e+08-7.550000e-03j -0.0000-0.0

In [48]:
# Using GridSearchCV
ridge_cv=RidgeCV(alphas=penelty, cv=5)
model = ridge_cv.fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
print("Best Alpha:{0:.5f}, R2:{1:.4f}".format(model.alpha_, model.best_score_))


Best Alpha:0.90000, R2:0.4419


0.9

In [60]:
model_best = Ridge(alpha=model.alpha_).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
score = model_best.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
pred_y = model_best.predict(X_scal.iloc[valid_idx])
mse = np.sqrt(mean_squared_error(Y.iloc[valid_idx],pred_y))
print(f"Alpha:{model.alpha_:.5f}, R2:{score:.7f}, MSE:{mse:.7f}, RMSE:{np.sqrt(mse):.7f}")
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scal.columns)

Alpha:0.90000, R2:0.5263592, MSE:55.7637783, RMSE:7.4675149
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   42.643650  3.819412e+08+1.308048e+00j  0.0000-0.0000j  1.000000
AGE         -10.492753  2.379306e+01+1.368090e-01j -0.4410+0.0025j  0.659939
BMI         121.677019  3.185560e+01+1.175510e-01j  3.8196-0.0141j  0.000205
BP           66.649695  2.809700e+01+2.136490e-01j  2.3720-0.0180j  0.019133
S1          -21.091765  1.642265e+02+2.031570e-01j -0.1284+0.0002j  0.898004
S2           -5.450923  1.152949e+02+5.286150e-01j -0.0473+0.0002j  0.962363
S3          -55.208206  7.328338e+01-1.523860e-01j -0.7533-0.0016j  0.452581
S4           17.858217  5.733621e+01+1.527930e-01j  0.3115-0.0008j  0.755939
S5          119.080753  5.044998e+01+5.800700e-02j  2.3604-0.0027j  0.019721
S6           22.603065  3.410336e+01+2.433040e-01j  0.6627-0.0047j  0.508637
SEX_1        10.489062  3.819412e+08+1.290408e+00j  0.0000-0.0000j  1.000000
SE

In [57]:
# GridSearchCV Result
model_best = Ridge(alpha=model.alpha_).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
score = model_best.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
pred_y = model_best.predict(X_scal.iloc[valid_idx])
mse = np.sqrt(mean_squared_error(Y.iloc[valid_idx], pred_y))
print("Alpha:{0:.5f}, R2:{1:.7f}, MSE:{2:.7f}, RMSE:{3:.7f}".format(0.01, score, mse, np.sqrt(mse)))
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scal.columns)

Alpha:0.01000, R2:0.5263592, MSE:55.7637783, RMSE:7.4675149
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   42.643650  3.819412e+08+1.308048e+00j  0.0000-0.0000j  1.000000
AGE         -10.492753  2.379306e+01+1.368090e-01j -0.4410+0.0025j  0.659939
BMI         121.677019  3.185560e+01+1.175510e-01j  3.8196-0.0141j  0.000205
BP           66.649695  2.809700e+01+2.136490e-01j  2.3720-0.0180j  0.019133
S1          -21.091765  1.642265e+02+2.031570e-01j -0.1284+0.0002j  0.898004
S2           -5.450923  1.152949e+02+5.286150e-01j -0.0473+0.0002j  0.962363
S3          -55.208206  7.328338e+01-1.523860e-01j -0.7533-0.0016j  0.452581
S4           17.858217  5.733621e+01+1.527930e-01j  0.3115-0.0008j  0.755939
S5          119.080753  5.044998e+01+5.800700e-02j  2.3604-0.0027j  0.019721
S6           22.603065  3.410336e+01+2.433040e-01j  0.6627-0.0047j  0.508637
SEX_1        10.489062  3.819412e+08+1.290408e+00j  0.0000-0.0000j  1.000000
SE